# **Used Car Price Predictor Notebook**

### import required libraries

In [1]:
import keras 
import pandas as pd
import pandasql as sqldf
from zip_to_state import zip_to_state, state_to_abbrev
import plotly.express as px
import numpy as np
from scipy import stats
import seaborn as sns


## **Phase 1: Data Collection**
#### **Step 1: Extract Training Data from Parquet File**

In [2]:
# time to run: 4m 57s
df = pd.read_csv('used_cars_data.csv')

C:\Users\Daymo\AppData\Local\Temp\ipykernel_15756\2680727636.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('used_cars_data.csv')


# ---------------------------------------------------------------------------

#### **Step 2: Preview the Dataset's Schema**

In [19]:
pd.reset_option('all')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head(2)
# df.shape
# df.columns


,body_type,daysonmarket,dealer_zip,exterior_color,fleet,frame_damaged,has_accidents,horsepower,is_new,listing_color,make_name,mileage,model_name,owner_count,price,salvage,trim_name,wheel_system,year
0,SUV / Crossover,522,960,Solar Yellow,NaN,NaN,NaN,177.0,True,YELLOW,Jeep,7.0,Renegade,NaN,23141.0,NaN,Latitude FWD,FWD,2019
1,SUV / Crossover,207,922,Narvik Black,NaN,NaN,NaN,246.0,True,BLACK,Land Rover,8.0,Discovery Sport,NaN,46500.0,NaN,S AWD,AWD,2020


# ---------------------------------------------------------------------------

## **MLOPs Phase 2: Data Ingestion**
#### For the purposes of this tutorial, we will not be ingesting data. However, in an organizational setting, you'll use an ingestion framework, such as Apache Airflow, Apache Spark, or Apache Kafka, to collect data into a Data Lake to create a curated Training Dataset.

## **MLOPs Phase 3: Data Storage **
#### Since we already extracted this data was previously extracted from BigQuery, this phase has already been completed. Typically, you'll use a Cloud Storage Bucket (e.g., AWS S3, Google Cloud Storage) to store all data to be used in ML applications.

# ---------------------------------------------------------------------------

## **MLOps Phase 5: Exploratory Data Analysis (EDA)**

#### **Find the total number of missing values per column**

In [9]:
for col in df.columns:
    # Count rows with NaN as 'mileage'
    nan_mileage_count = df[col].isna().sum()

    # Print the count
    print(f"{col}: {nan_mileage_count}")
    

body_type: 13543
daysonmarket: 0
dealer_zip: 0
exterior_color: 49955
fleet: 1426595
frame_damaged: 1426595
has_accidents: 1426595
horsepower: 172386
is_new: 0
listing_color: 0
make_name: 0
mileage: 144387
model_name: 0
owner_count: 1517013
price: 0
salvage: 1426595
trim_name: 116294
wheel_system: 146732
year: 0


#### **Catagorical Data Col Analysis**

In [10]:
categorical_columns = ['body_type', 'listing_color','has_accidents','is_new','salvage','wheel_system','make_name', 'frame_damaged','fleet']
#'exterior_color'
unique_value_counts = df[categorical_columns].nunique()

value_frequencies = {}
for col in categorical_columns:
    value_frequencies[col] = df[col].value_counts()

print("Count of Unique Values:")
print(unique_value_counts)


Count of Unique Values:
body_type          9
listing_color     15
has_accidents      2
is_new             2
salvage            2
wheel_system       5
make_name        100
frame_damaged      2
fleet              2
dtype: int64


In [11]:
# Loop through categorical columns and create histograms
for col, frequencies in value_frequencies.items():
    fig = px.bar(frequencies, x=frequencies.index, y=frequencies.values, labels={'x': col, 'y': 'Frequency'})
    
    # Customize the layout if needed
    fig.update_layout(
        title=f'Histogram of {col}',
        xaxis_title=col,
        yaxis_title='Frequency',
        xaxis={'categoryorder':'total descending'}  # Sort categories by frequency
    )
    
    fig.show()  

#### **Numerical Data Col Analysis**

In [ ]:

numerical_columns = [ 
       'horsepower','daysonmarket', 'mileage', 'owner_count', 'price','year',] 

summary_statistics = df[numerical_columns].describe()
rounded_summary_statistics = summary_statistics.round(2)  
print(rounded_summary_statistics)


In [14]:
df = zip2State(df)

In [ ]:

x_range_price = [0, 200000]
x_range_year = [1995, 2022]
x_range_mileage = [0, 100000]

fig_price = px.histogram(df, x='price', title='Price Distribution - Outliers removed', range_x=x_range_price,  nbins=500)
fig_mileage = px.histogram(df, x='mileage', title='Mileage Distribution - Outliers removed', range_x=x_range_mileage,  nbins=32000)
fig_year = px.histogram(df, x='year', title='Year Distribution - Outliers removed', range_x=x_range_year)

state_summary = pd.DataFrame(df['dealer_zip'].astype(str).str[:5])
state_summary['state'] = state_summary['dealer_zip'].astype(int).apply(
    lambda zip_code: next((state for (zipcode_range, state) in zip_to_state.items() if zip_code >= zipcode_range[0] and zip_code <= zipcode_range[1]), 'Unknown')
)
state_summary = state_summary.groupby('state').size().reset_index(name='total_listings')
state_summary['state_abbr'] = state_summary['state'].map(state_to_abbrev)
state_summary = state_summary[state_summary['state_abbr'] != 'Unknown']
max_value = state_summary['total_listings'].max()
fig_map = px.choropleth(
    state_summary,
    locations='state_abbr',  
    locationmode='USA-states',      
    color='total_listings',       
    color_continuous_scale='greens',
    range_color=(0, max_value), 
    scope='usa'                  
)

fig_map.update_geos(fitbounds='locations', visible=False)
fig_map.show()

fig_price.show()
fig_mileage.show()
fig_year.show()


#### **Find total outliers per col and how many standard deviations out**

In [32]:

# # Assuming df is your DataFrame with columns "mileage," "owner_count," "price," and "year"
columns_of_interest = [ 'mileage', 'year','horsepower','price']
# Initialize a dictionary to store outlier counts for each column
outlier_counts = {}

# Set the Z-score threshold for identifying outliers
z_score_threshold = 10  # You can adjust this threshold as needed

for col in columns_of_interest:
    z_scores = np.abs(stats.zscore(df[col]))
    outliers = (z_scores > z_score_threshold)
    outlier_count = outliers.sum()
    outlier_counts[col] = outlier_count

#Display the counts of outliers for each column
for col, count in outlier_counts.items():
    print(f"{col}: {count} outliers")

# Identify outliers for the "price" column
price_z_scores = np.abs(stats.zscore(df['price']))
price_outliers = (price_z_scores > z_score_threshold)
# Filter the DataFrame to get rows where "price" is an outlier
price_outlier_rows = df[price_outliers]
desired_columns = ['model_name', 'year', 'mileage', 'price']
price_outlier_rows = price_outlier_rows[desired_columns]
# Print the rows that are outliers for the "price" column
print("Rows where 'price' is an outlier:")
print(price_outlier_rows)

mileage: 0 outliers
year: 6056 outliers
horsepower: 0 outliers
price: 5288 outliers
Rows where 'price' is an outlier:
                       model_name  year   mileage      price
2948                     Bentayga  2019   13117.0   159990.0
2980                      G-Class  2020    5100.0   207900.0
3077                           R8  2018    1256.0   159999.0
4778                      Huracan  2018   32076.0   169950.0
6764                           M8  2021       0.0   148645.0
6798                           M8  2020       0.0   151195.0
6814                           M8  2020       0.0   156695.0
6824                           M8  2020       0.0   156695.0
6851                           M8  2020       0.0   157195.0
6856                           M8  2020       0.0   157195.0
6891                           M8  2020       0.0   157195.0
8891                           M8  2021       0.0   152295.0
10665                       Ghost  2016   10120.0   155995.0
14765                       

In [33]:
def custom_format(x):
    if x >= 1e6:
        return '{:.4g} mil'.format(x / 1e6)
    elif x >= 1e3:
        return '{:.4g}k'.format(x / 1e3)
    else:
        return '{:.4g}'.format(x)

# # Apply the custom formatting function to specific columns using applymap
# columns_to_format = ['price', 'daysonmarket','horsepower','mileage','owner_count']  # Add the columns you want to format
# price_outlier_rows[columns_to_format] = price_outlier_rows[columns_to_format].applymap(custom_format)

# Assuming you want to generate summary statistics only for the "price" column
price_outlier_rows[['price']].sort_values(by='price', ascending=False).applymap(custom_format)



,price
1410578,3.3 mil
174408,3.195 mil
2786672,2.75 mil
2795480,2.739 mil
230048,2.699 mil
2178397,2.25 mil
174397,2.195 mil
2178384,2.175 mil
2862805,2 mil
2057024,1.996 mil


#### **Create Correlation Matrix**

In [ ]:
corr_matrix = df[['mileage', 'price', 'year', 'horsepower', 'owner_count','daysonmarket']].corr()
print(corr_matrix)
plt = sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')


## **DataPipline (Preparation and Wrangling)**

# ---------------------------------------------------------------------------
#### **Step 1: Use Intuition to Remove Irrelevant Features**

In [3]:
columns_to_be_removed = ['power',"latitude","longitude","city","vin","back_legroom","bed","bed_height","bed_length","cabin","city_fuel_economy","combine_fuel_economy","description","engine_cylinders","engine_displacement","engine_type","franchise_dealer","franchise_make","front_legroom","fuel_tank_volume","fuel_type","height","highway_fuel_economy","interior_color","isCab","is_certified","is_cpo","is_oemcpo","length","listed_date","listing_id","main_picture_url","maximum_seating","savings_amount","seller_rating","sp_id","sp_name","theft_title","torque","transmission","transmission_display","vehicle_damage_category","wheel_system_display","wheelbase","width",'trimId', 'major_options']
# columns_to_be_removed = ["exterior_color"]
df = df.drop(columns=columns_to_be_removed)

In [13]:
df_cleaned.head(2)


,dealer_zip,exterior_color,fleet,frame_damaged,has_accidents,is_new,listing_color,make_name,mileage,model_name,owner_count,price,salvage,trim_name,wheel_system,year
0,960,Solar Yellow,NaN,NaN,NaN,True,YELLOW,Jeep,7.0,Renegade,NaN,23141.0,NaN,Latitude FWD,FWD,2019
1,922,Narvik Black,NaN,NaN,NaN,True,BLACK,Land Rover,8.0,Discovery Sport,NaN,46500.0,NaN,S AWD,AWD,2020


#### **Remove outlier from major columns**

#### **Convert all data in each column to target type**

In [ ]:

data_types = {
    'fleet': bool,
    'frame_damaged': bool,
    'has_accidents': bool,
    'is_new': bool,
    'mileage': int,
    'owner_count': int,
    'price': int,
    'salvage': bool,
    'year': int,
    'power':str
}

# Use the astype method to convert columns to the specified data types
df = df.astype(data_types)

# If you want to convert 'listing_color', 'make_name', 'model_name',  'trim_name', and 'wheel_system' columns to string
string_columns = ['listing_color', 'make_name', 'model_name', 'trim_name', 'wheel_system']
df[string_columns] = df[string_columns].astype(str)


#### **get rid of redundant year values in trim name col**

In [ ]:

df_cleaned['trim_name'] = df_cleaned['trim_name'].str.replace(r'20\d{2}\.5 ', '', regex=True)

#### **Simple data imputation to fill all the missing mileage values to the average mileage of vehicles of theat same year**

In [ ]:
# Calculate the average mileage for each year
average_mileage_by_year = df.groupby('year')['mileage'].transform('mean')

# Fill NaN values in 'mileage' with the corresponding average for the year
df['mileage'].fillna(average_mileage_by_year, inplace=True)
df.dropna(subset=['mileage'], inplace=True)
# Print the updated DataFrame
print(df)





# ---------------------------------------------------------------------------

## **MLOps Phase 6: Feature Engineering**

#### **Turn zip into a state field and only keep US states**

In [ ]:
def zip2State(df):
    df['dealer_zip'] = df['dealer_zip'].astype(str).str[:5]
    df['state'] = df['dealer_zip'].astype(int).apply(
    lambda zip_code: next((state for (zipcode_range, state) in zip_to_state.items() if zip_code >= zipcode_range[0] and zip_code <= zipcode_range[1]), 'Unknown')
)
    return df
df_cleaned = df_cleaned.drop(columns=['dealer_zip'])

#### **Step 3: Create DataFrame that Merges Pre-Preprocessed Data and Preprocessed Data**

# ---------------------------------------------------------------------------

## **MLOps Phase 7: Model Development**

#### **Step 1: Identify Features and Label**

#### **Step 2: Split Training Data into Training and Evaluation Subsets**

#### **Step 3: Fit Training Subset to Model**

#### **Step 4: Show Feature Importance Metrics**